In [24]:
# importing the necessary packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import csv

In [25]:
search_term1="COVID"
search_term2="stroke"
url="https://pubmed.ncbi.nlm.nih.gov/?term=%22{}%22+%22{}%22&size=200".format(search_term1,search_term2)
url2="https://pubmed.ncbi.nlm.nih.gov/"

r1 = requests.get(url)
all_papers = r1.content

soup1 = BeautifulSoup(all_papers, 'html5lib')

papers = soup1.find_all('a', class_='labs-docsum-title')

link_abstracts=[]
link2=url2+papers[1]['href']
r2 = requests.get(link2)
paper_content = r2.content
soup2 = BeautifulSoup(paper_content, 'html5lib')
title=soup2.find_all('title')

In [26]:
# from https://medium.com/@jorlugaqui/how-to-strip-html-tags-from-a-string-in-python-7cb81a2bbf44
def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def remove_line(text):
    text_new = text.replace('\n','')
    return text_new

In [27]:
for paper in papers:
    link=url2+paper['href']
    r2 = requests.get(link)
    paper_content = r2.content
    soup2 = BeautifulSoup(paper_content, 'html5lib')
    abstract = soup2.find_all('div', class_='abstract-content')
    title=soup2.find_all('title')
    if not bool(abstract):
        continue
        
    raw_text = str(abstract)
    raw_text2 = remove_html_tags(raw_text)
    abstract_text=remove_line(raw_text2)
    raw_text = str(title)
    raw_text2 = remove_html_tags(raw_text)
    title_text=remove_line(raw_text2)
    
    link_abstracts.append(tuple([link, title_text, abstract_text]))

In [28]:
print(link_abstracts)
filename="test1"
file_ext=filename+".csv"

with open(file_ext, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['link','title','abstract'])
        for row in link_abstracts:
            writer.writerow(row)

[('https://pubmed.ncbi.nlm.nih.gov//32493597/?from_term=%22COVID%22+%22stroke%22&from_size=200&from_pos=2', '[Stroke as a Complication and Prognostic Factor of COVID-19 - PubMed]', '[                                              Introduction:                    Contradictory data have been reported on the incidence of stroke in patients with COVID-19 and the risk of SARS-CoV-2 infection among patients with history of stroke.                                                        Methods:                    This study systematically reviews case series reporting stroke as a complication of COVID-19, and analyses the prognosis of patients with COVID-19 and history of stroke. The pathophysiological mechanisms of stroke in patients with COVID-19 are also reviewed.                                                        Conclusions:                    History of stroke increases the risk of death due to COVID-19 by 3 times. Stroke currently seems not to be one of the main complications of CO